In [1]:
import solopy
from solopy import _utils
import ccdproc
from astropy.io import fits
from astropy.time import Time
from pathlib import Path
import logging
import bz2
import shutil # 임시 디렉터리 삭제용

In [2]:
subdir = "2025_0717" # observation date

In [3]:
# Data directory
DATADIR = Path("../solo-data")

# Lv0 data directory
LV0DIR = DATADIR/"Lv0"
LV0_subdir = LV0DIR / subdir

# Lv1 data directory
LV1DIR = DATADIR/"Lv1"
LV1_subdir = LV1DIR / subdir
LV1_subdir.mkdir(parents=True, exist_ok=True)

# Master calibration files directory
MASTERDIR = DATADIR / "calibration_files"
MASTERDIR.mkdir(parents=True, exist_ok=True)

# Log directory
LOGDIR = DATADIR/"log"
LOGDIR.mkdir(parents=True, exist_ok=True)

fpath_log_lv0 = LOGDIR/f'lv0_{subdir}.log' # log file path for Lv0
if fpath_log_lv0.exists():
    fpath_log_lv0.unlink() # remove existing log file
    
fpath_log_lv1 = LOGDIR/f'lv1_{subdir}.log' # log file path for Lv1
if fpath_log_lv1.exists():
    fpath_log_lv1.unlink() # remove existing log file

fpath_log_comb = LOGDIR/f'combmaster_{subdir}.log' # log file path for CombMaster
if fpath_log_comb.exists():
    fpath_log_comb.unlink() # remove existing log file

In [4]:
# Logger
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[
                        logging.StreamHandler(),
                        logging.FileHandler(LOGDIR/"solo-example.log")
                    ])
logger = logging.getLogger(__name__)

In [5]:
# Step 1: Batch Decompress
logger.info("--- Step 1: Batch Decompress ---")
_utils.batch_decompress(LV0_subdir, LV0_subdir, delete_source=True)

2025-10-27 20:23:33,802 [INFO] --- Step 1: Batch Decompress ---
2025-10-27 20:23:33,803 [INFO] Decompressing 147 files from ../solo-data/Lv0/2025_0717 to ../solo-data/Lv0/2025_0717...
Decompressing: 100%|██████████| 147/147 [01:22<00:00,  1.78it/s]


In [6]:
# Step 2: Lv0 Header Update
logger.info("--- Step 2: Lv0 Header Update ---")
lv0 = solopy.Lv0(log_file=fpath_log_lv0)

fpaths_fits = list(LV0_subdir.glob("*.fits"))
if not fpaths_fits:
    raise FileNotFoundError(f"No .fits files found in {LV0_subdir}.")

for fpath in fpaths_fits:
    lv0.update_header(fpath) # Update header in-place

2025-10-27 20:25:46,881 [INFO] --- Step 2: Lv0 Header Update ---
2025-10-27 20:25:46,953 [INFO] Updated LV0 header: dk03_007.fits
2025-10-27 20:25:46,994 [INFO] Updated LV0 header: dk02_007.fits
2025-10-27 20:25:47,037 [INFO] Updated LV0 header: dark008_002.fits
2025-10-27 20:25:47,082 [INFO] Updated LV0 header: bias_006.fits
2025-10-27 20:25:47,127 [INFO] Updated LV0 header: ELENORA_031_20250717061504.fits
2025-10-27 20:25:47,170 [INFO] Updated LV0 header: ELENORA_032_20250717061637.fits
2025-10-27 20:25:47,212 [INFO] Updated LV0 header: CARLOVA_001_20250717090310.fits
2025-10-27 20:25:47,254 [INFO] Updated LV0 header: dark004_009.fits
2025-10-27 20:25:47,295 [INFO] Updated LV0 header: dark090_002.fits
2025-10-27 20:25:47,336 [INFO] Updated LV0 header: dk01_002.fits
2025-10-27 20:25:47,377 [INFO] Updated LV0 header: CARLOVA_012_20250717092002.fits
2025-10-27 20:25:47,419 [INFO] Updated LV0 header: ELENORA_014_20250717054859.fits
2025-10-27 20:25:47,462 [INFO] Updated LV0 header: CARLO

In [7]:
logger.info("--- Step 3: CombMaster Frame Generation ---")
comb = solopy.CombMaster(log_file=fpath_log_comb)

# Collect all Lv0 fits files
all_lv0_fits = ccdproc.ImageFileCollection(LV0_subdir, glob_exclude="*.bz2", glob_include="*.fits")

# Master bias
bias_files = all_lv0_fits.files_filtered(IMAGETYP="BIAS", include_path=True)
if bias_files:
    comb.make_mbias(bias_files, MASTERDIR)
    
# Master dark
dark_files = all_lv0_fits.files_filtered(IMAGETYP="DARK", include_path=True)
if dark_files:
    comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')

2025-10-27 20:25:57,412 [INFO] --- Step 3: CombMaster Frame Generation ---
2025-10-27 20:25:57,621 [INFO] Starting master bias combination...
2025-10-27 20:25:57,622 [INFO] Loading 8 frames...
2025-10-27 20:25:57,640 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,658 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,674 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,691 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,711 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,732 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:25:57,752 [INFO] using the unit adu passed to the FITS reader instead of the unit adu

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: splitting each image into 10 chunks to limit memor

2025-10-27 20:26:08,390 [INFO] Master bias saved to kl4040.bias.comb.20250717.fits
2025-10-27 20:26:08,509 [INFO] Starting master dark creation...
2025-10-27 20:26:08,527 [INFO] Loading 81 frames...
2025-10-27 20:26:08,545 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,562 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,580 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,639 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,666 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,682 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,699 [INFO] using the unit adu passed to the FITS reader instead of the un

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


2025-10-27 20:26:08,752 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,770 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,786 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,803 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,821 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,839 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,857 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,874 [INFO] using the unit adu passed to the FITS reader instead of the unit adu in the FITS file.
2025-10-27 20:26:08,892 [INFO] using the unit adu passed

...

2025-10-27 20:27:43,892 [INFO] Master dark saved to kl4040.dark.2s.comb.20250717.fits
2025-10-27 20:27:43,892 [INFO] Combining 9 darks for exptime 3.0s...
2025-10-27 20:27:43,937 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 20:27:56,534 [INFO] Master dark saved to kl4040.dark.3s.comb.20250717.fits
2025-10-27 20:27:56,534 [INFO] Combining 9 darks for exptime 5.0s...
2025-10-27 20:27:56,581 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 20:28:09,332 [INFO] Master dark saved to kl4040.dark.5s.comb.20250717.fits


In [ ]:

fpath_log_lv0 = LOGDIR/'lv0_example.log' # log file path for Lv0
if fpath_log_lv0.exists():
    fpath_log_lv0.unlink() # remove existing log file

# Data directory

MASTERDIR = DATADIR / "calibration_files"
MASTERDIR.mkdir(parents=True, exist_ok=True)

subdir = "2025_0717"

# Lv0 Data directory
Lv0DIR = DATADIR / "Lv0"
Lv0_subdir = Lv0DIR / subdir

# Lv1 Data directory
Lv1DIR = DATADIR / "Lv1"
Lv1_subdir = Lv1DIR / subdir  
Lv1_subdir.mkdir(parents=True, exist_ok=True)

Lv1_wcsdir = DATADIR / "Lv1_wcs"
Lv1_wcsdir.mkdir(parents=True, exist_ok=True)

In [3]:
# Initialize Lv0 class
lv0 = solopy.Lv0(log_file=fpath_log_lv0)

# Initialize Lv1 class
lv1 = solopy.Lv1(log_file=fpath_log_lv1)

# Initialize CombMaster class
comb = solopy.CombMaster(log_file=fpath_log_comb)

In [4]:
# solopy 로거 설정 (이미 클래스 내부에 설정되어 있지만, 메인 스크립트의 로그도 함께 보려면 설정)
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[
                        logging.StreamHandler(), # 콘솔 출력
                        logging.FileHandler(LOGDIR/"solopy-example.log") # 파일 출력
                    ])
logger = logging.getLogger(__name__)

In [5]:
bias_files = []
dark_files = []
flat_files = [] # 타겟 필터와 일치하는 플랫
science_files = [] # 타겟 필터와 일치하는 라이트
all_files = [] # 모든 파일

logger.info(f"Scanning {Lv0_subdir} for file types...")
all_files_count = 0

for fpath in Lv0_subdir.glob("*.fits.bz2"):
    all_files.append(fpath)

    all_files_count += 1
    fobj = None
    
    try:
        
        # bz2 파일을 열고 'getheader'로 헤더만 빠르게 읽기
        fobj = bz2.BZ2File(fpath, "rb")
        hdr = fits.getheader(fobj) 
        
        imagetyp = hdr.get('IMAGETYP', 'UNKNOWN').upper()
        
        if imagetyp == 'BIAS':
            bias_files.append(fpath)
        elif imagetyp == 'DARK':
            dark_files.append(fpath)
        elif imagetyp == 'LIGHT':
            science_files.append(fpath)
            
    except Exception as e:
        logger.warning(f"Could not read header from {fpath.name}: {e}")
        logger.warning(f"Using {fpath.name} as a fallback.")
        
    finally:
        if fobj:
            fobj.close()

logger.info(f"Scanned {all_files_count} files. Found:")
logger.info(f"{len(bias_files)} BIAS")
logger.info(f"{len(dark_files)} DARK")
logger.info(f"{len(science_files)} LIGHT (Science)")

2025-10-27 18:32:00,321 [INFO] Scanning ../solo-data/Lv0/2025_0717 for file types...
2025-10-27 18:33:15,473 [INFO] Scanned 147 files. Found:
2025-10-27 18:33:15,473 [INFO] 8 BIAS
2025-10-27 18:33:15,474 [INFO] 81 DARK
2025-10-27 18:33:15,474 [INFO] 58 LIGHT (Science)


In [ ]:
# Update header (LV0)
if all_files:
    logger.info("--- Updating Lv0 headers ---")
    for f in all_files:
        lv0.update_header(f)

2025-10-27 15:52:38,808 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_002.fits.bz2
2025-10-27 15:52:38,808 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_002.fits.bz2
2025-10-27 15:52:41,210 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_003.fits.bz2
2025-10-27 15:52:41,210 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_003.fits.bz2
2025-10-27 15:52:43,724 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/CARLOVA_006_20250717091050.fits.bz2
2025-10-27 15:52:43,724 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/CARLOVA_006_20250717091050.fits.bz2
2025-10-27 15:52:46,121 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dark007_001.fits.bz2
2025-10-27 15:52:46,121 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dark007_001.fits.bz2
2025-10-27 15:52:48,667 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/ELENORA_010_20250717054251.fits.bz2
2025-10-27 15:52:48,667 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_071

In [ ]:
# Master bias creation
logger.info("--- Starting Master Bias ---")
if bias_files:
    comb.make_mbias(bias_files, MASTERDIR)
else:
    logger.warning("No BIAS files found to combine.")

In [6]:
# Master dark creation
logger.info("--- Starting Master Dark ---")
if dark_files:
    comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')
else:
    logger.warning("No DARK files found to combine.")

2025-10-27 18:33:23,264 [INFO] --- Starting Master Dark ---
2025-10-27 18:33:23,264 [INFO] Starting master dark creation...
2025-10-27 18:33:23,264 [INFO] Starting master dark creation...
2025-10-27 18:33:23,284 [INFO] Loading 81 frames...
2025-10-27 18:33:23,284 [INFO] Loading 81 frames...
2025-10-27 18:35:35,025 [INFO] Using master bias: kl4040.bias.comb.20250717.fits
2025-10-27 18:35:35,025 [INFO] Using master bias: kl4040.bias.comb.20250717.fits
2025-10-27 18:35:38,621 [INFO] Combining 9 darks for exptime 3.0s...
2025-10-27 18:35:38,621 [INFO] Combining 9 darks for exptime 3.0s...
2025-10-27 18:35:38,737 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 18:35:51,007 [INFO] Master dark saved to kl4040.dark.3s.comb.20250717.fits
2025-10-27 18:35:51,007 [INFO] Master dark saved to kl4040.dark.3s.comb.20250717.fits
2025-10-27 18:35:51,008 [INFO] Combining 9 darks for exptime 7.0s...
2025-10-27 18:35:51,008 [INFO] Combining 9 darks for exptime 7.0s...
2025-10-27 18:35:51,090 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 18:36:03,408 [INFO] Master dark saved to kl4040.dark.7s.comb.20250717.fits
2025-10-27 18:36:03,408 [INFO] Master dark saved to kl4040.dark.7s.comb.20250717.fits
2025-10-27 18:36:03,410 [INFO] Combining 9 darks for exptime 4.0s...
2025-10-27 18:36:03,410 [INFO] Combining 9 darks for exptime 4.0s...
2025-10-27 18:36:03,469 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


...

2025-10-27 18:37:05,277 [INFO] Master dark saved to kl4040.dark.90s.comb.20250717.fits
2025-10-27 18:37:05,277 [INFO] Master dark saved to kl4040.dark.90s.comb.20250717.fits
2025-10-27 18:37:05,278 [INFO] Combining 9 darks for exptime 8.0s...
2025-10-27 18:37:05,278 [INFO] Combining 9 darks for exptime 8.0s...
2025-10-27 18:37:05,330 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 18:37:18,139 [INFO] Master dark saved to kl4040.dark.8s.comb.20250717.fits
2025-10-27 18:37:18,139 [INFO] Master dark saved to kl4040.dark.8s.comb.20250717.fits
2025-10-27 18:37:18,140 [INFO] Combining 9 darks for exptime 1.0s...
2025-10-27 18:37:18,140 [INFO] Combining 9 darks for exptime 1.0s...
2025-10-27 18:37:18,198 [INFO] splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 11 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 18:37:30,734 [INFO] Master dark saved to kl4040.dark.1s.comb.20250717.fits
2025-10-27 18:37:30,734 [INFO] Master dark saved to kl4040.dark.1s.comb.20250717.fits


In [5]:
bias_files = list(Lv0_subdir.glob("*bias*.fits.bz2"))
if bias_files:
    comb.make_mbias(bias_files, MASTERDIR)
else:
    logger.warning("No bias files found to combine.")

2025-10-27 17:41:50,831 [INFO] Starting master bias combination...
2025-10-27 17:41:50,831 [INFO] Starting master bias combination...
2025-10-27 17:41:50,832 [INFO] Loading 8 frames...
2025-10-27 17:41:50,832 [INFO] Loading 8 frames...
2025-10-27 17:41:58,638 [INFO] Combining 8 bias CCDData objects...
2025-10-27 17:41:58,638 [INFO] Combining 8 bias CCDData objects...
2025-10-27 17:41:58,667 [INFO] splitting each image into 10 chunks to limit memory usage to 500000000.0 bytes.


INFO: splitting each image into 10 chunks to limit memory usage to 500000000.0 bytes. [ccdproc.combiner]


2025-10-27 17:42:08,851 [INFO] Master bias saved to kl4040.bias.comb.20250717.fits
2025-10-27 17:42:08,851 [INFO] Master bias saved to kl4040.bias.comb.20250717.fits


In [ ]:
# 2-2. 마스터 다크 생성
# Lv0 디렉터리에서 IMAGETYP='DARK'인 파일 찾기 (파일명으로 검색)
logger.info("--- Starting Master Dark ---")
dark_files = list(Lv0_subdir.glob("*dark*.fits.bz2")) + \
                list(Lv0_subdir.glob("*dk*.fits.bz2"))
if dark_files:
    comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')
else:
    logger.warning("No dark files found to combine.")

In [ ]:
import solopy
from pathlib import Path




# --- 1. 경로 설정 ---
# (경로는 예시이므로 실제 환경에 맞게 수정하세요)
BASE_DIR = Path(".") # 현재 스크립트가 실행되는 위치
DATADIR = Path("../solo-data") # 데이터 상위 폴더

# Lv0 헤더가 업데이트된 파일이 있는 폴더
Lv0_subdir = DATADIR / "Lv0" / "2025_0717"


# 사용할 필터 (예: 'R' 필터)
TARGET_FILTER = 'R' # 이 필터 이름은 파일명 검색 및 mflat 생성에 사용됩니다.

logger.info(f"Using Lv0 data from: {Lv0_subdir}")
logger.info(f"Master frames will be saved to: {MASTERDIR}")
logger.info(f"Lv1 WCS output to: {Lv1_wcsdir}")
logger.info(f"Lv1 Processed output to: {LV1_PROC_DIR}")

# --- 2. CombMaster: 마스터 프레임 생성 ---
# CombMaster 초기화 (solopy 내부 로거 사용)
comb = solopy.CombMaster(log_file="combmaster.log")




# --- 3. Lv1: 과학 프레임 처리 ---
# Lv1 초기화 (solopy 내부 로거 사용)
lv1 = solopy.Lv1(log_file="lv1_processing.log")

# 3-1. 처리할 과학(Light) 프레임 목록 가져오기
# (파일명에 bias, dark, dk, flat이 포함되지 않은 모든 파일)
logger.info("--- Starting Lv1 Processing ---")
science_files = [f for f in Lv0_subdir.glob("*.fits.bz2")
                 if 'bias' not in f.name.lower() and \
                    'dark' not in f.name.lower() and \
                    'dk' not in f.name.lower() and \
                    'flat' not in f.name.lower()]

if not science_files:
    logger.warning("No science files found in Lv0 directory.")
else:
    logger.info(f"Found {len(science_files)} science frames for Lv1 processing.")

    # 3-2. WCS 업데이트
    # (Lv0 파일 -> Lv1_WCS_DIR)
    wcs_updated_files = []
    for fpath in science_files:
        try:
            logger.info(f"Processing WCS for: {fpath.name}")
            wcs_path = lv1.update_wcs(fpath, Lv1_wcsdir)
            if wcs_path:
                wcs_updated_files.append(wcs_path)
        except Exception as e:
            logger.error(f"Failed WCS update for {fpath.name}: {e}")

    # 3-3. 전처리 (Bias, Dark, Flat 보정)
    # (Lv1_WCS_DIR 파일 -> Lv1_PROC_DIR)
    if not wcs_updated_files:
         logger.warning("No WCS files to process (WCS step failed or produced no files).")
    else:
        logger.info(f"Starting preprocessing for {len(wcs_updated_files)} WCS-updated files...")
        for fpath_wcs in wcs_updated_files:
            try:
                logger.info(f"Preprocessing: {fpath_wcs.name}")
                lv1.preprocessing(fpath_wcs, LV1_PROC_DIR, MASTERDIR)
            except Exception as e:
                logger.error(f"Failed preprocessing for {fpath_wcs.name}: {e}")

logger.info("--- Pipeline Finished ---")